In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

# Load data

In [2]:
#This should be in "data" folder inside "viz"
trade_mx = pd.read_parquet(r'../data/trade_matrix_carbon_footprint_v4.parquet')

In [3]:
coords = pd.read_csv(r"../data/country_to_continent_codes.csv")
coords.head()

,Area Code,Continent,Country,Capital,Latitude,Longitude
0,1,europe,armenia,yerevan,40.183300,44.516700
1,2,asia,afghanistan,kabul,34.526000,69.181000
2,3,europe,albania,tirana,41.327500,19.818900
3,4,africa,algeria,algiers,36.737232,3.086472
4,7,africa,angola,luanda,-8.839988,13.289437


This to aggregate items in items groups

In [4]:
new_items_df = pd.read_csv(r'../data/food_codes_categorized_final.csv', encoding = 'ISO-8859-1')
new_items_df.dropna(subset='Item_emissions_table', inplace=True)
new_items_df['Item_code_emissions_table'] = new_items_df['Item_code_emissions_table'].astype('int64')

Select 2021 trade data

In [5]:
trade_mx = trade_mx[trade_mx['Year'] == 2021]
trade_mx.head()

,Reporter Country Code,Partner Country Code,Item Code,Year,Value_tons,Flag,distance_in_km,same_continent,share_border,any_island_or_missing,transportation_method,food_miles,kgCO2eq_tkm
22,10,2,221,2021,10.68,A,11355.422485,0,0.0,1,water,121275.912137,1212.759121
24,11,2,221,2021,3.77,A,4561.287724,0,0.0,0,water,17196.054720,171.960547
36,7,3,22,2021,15.75,A,5618.206482,0,0.0,0,water,88486.752092,884.867521
45,255,2,221,2021,1.26,A,5408.721719,0,0.0,0,water,6814.989366,68.149894
60,52,2,527,2021,0.50,A,1821.034476,0,0.0,0,water,910.517238,9.105172


## Aggregate items in trade mx

In [6]:
trade_mx = pd.merge(trade_mx, new_items_df, 
                               left_on='Item Code', right_on='Item_Code', how='inner')

In [7]:
cols_to_keep = ['Reporter Country Code',
 'Partner Country Code',
 'Value_tons',
 'kgCO2eq_tkm',
 'Item_Code',
 'Item',
 'Item_code_emissions_table',
 'Item_emissions_table']

trade_mx = trade_mx[cols_to_keep]
trade_mx.sample(5)

,Reporter Country Code,Partner Country Code,Value_tons,kgCO2eq_tkm,Item_Code,Item,Item_code_emissions_table,Item_emissions_table
69765,174,203,130.98,34863.995084,1069,Meat of ducks; fresh or chilled,1068,Ducks
17662,141,110,148.84,4475.827402,28,Husked rice,27,Rice
83546,79,223,336.68,6857.629339,16,Wheat and meslin flour,15,Wheat
83896,150,210,0.61,137.284838,976,Sheep,976,Sheep
27467,238,131,2160.00,150928.650620,898,Skim milk and whey powder,960,Cattle; dairy


## Merge with coordinates data

### Source:
- exporting country = Partner Country

### Target:
- importing country = Reporter Country

In [8]:
coors_cols = ['Area Code', 'Continent', 'Country', 'Latitude', 'Longitude']

trade_mx = trade_mx.merge(coords[coors_cols], left_on='Partner Country Code', right_on='Area Code', how='left')
trade_mx.drop('Area Code', axis=1, inplace=True)
trade_mx.rename(columns={'Continent': 'Continent_source',
                         'Country': 'Country_source',
                          'Latitude': 'lat_source',
                          'Longitude': 'lon_source'}, inplace=True)

In [9]:
trade_mx = trade_mx.merge(coords[coors_cols], left_on='Reporter Country Code', right_on='Area Code', how='left')
trade_mx.drop('Area Code', axis=1, inplace=True)
trade_mx.rename(columns={'Continent': 'Continent_target',
                            'Country': 'Country_target',
                            'Latitude': 'lat_target',
                            'Longitude': 'lon_target'}, inplace=True)

In [10]:
trade_mx.head()

,Reporter Country Code,Partner Country Code,Value_tons,kgCO2eq_tkm,Item_Code,Item,Item_code_emissions_table,Item_emissions_table,Continent_source,Country_source,lat_source,lon_source,Continent_target,Country_target,lat_target,lon_target
0,7,4,6.00,16468.361560,116,Potatoes,116,Potatoes,africa,algeria,36.737232,3.086472,africa,angola,-8.839988,13.289437
1,7,4,14.14,14645.448582,907,Processed cheese,960,Cattle; dairy,africa,algeria,36.737232,3.086472,africa,angola,-8.839988,13.289437
2,255,2,2.23,120.614494,176,Beans; dry,176,Beans; dry,asia,afghanistan,34.526000,69.181000,europe,belgium,50.850300,4.351700
3,1,9,0.47,62.964952,176,Beans; dry,176,Beans; dry,south america,argentina,-34.611800,-58.417300,europe,armenia,40.183300,44.516700
4,3,9,4128.00,479318.302685,238,Cake of soya beans,236,Soya beans,south america,argentina,-34.611800,-58.417300,europe,albania,41.327500,19.818900


# Map for Soya Beans (Crop item with highest travel and emissions)

In [11]:
def process_map_data(df, item_code):
    item_df = df[df['Item_code_emissions_table'] == item_code]
    cols_for_map = ['Reporter Country Code',
    'Partner Country Code',
    'lat_source',
    'lon_source',
    'Country_source',
    'Country_target',
    'Continent_target',
    'lat_target',
    'lon_target']

    item_df = item_df.groupby(cols_for_map)[['Value_tons', 'kgCO2eq_tkm']].sum().reset_index()
    emissions_mean = item_df['kgCO2eq_tkm'].mean()
    item_df = item_df[item_df['kgCO2eq_tkm'] > emissions_mean]

    return item_df

In [12]:
soya = process_map_data(trade_mx, 236)
soya

,Reporter Country Code,Partner Country Code,lat_source,lon_source,Country_source,Country_target,Continent_target,lat_target,lon_target,Value_tons,kgCO2eq_tkm
44,9,21,-15.8267,-47.9218,brazil,argentina,south america,-34.6118,-58.4173,218453.79,1.020960e+08
50,9,169,-25.2865,-57.6470,paraguay,argentina,south america,-34.6118,-58.4173,4601769.12,9.567707e+08
55,10,9,-34.6118,-58.4173,argentina,australia,oceania,-35.2809,149.1300,666038.00,7.814985e+07
83,11,79,52.5200,13.4050,germany,austria,europe,48.2082,16.3738,280218.90,2.934148e+07
89,11,106,41.9028,12.4964,italy,austria,europe,48.2082,16.3738,106506.47,1.627777e+07
...,...,...,...,...,...,...,...,...,...,...,...
2127,231,100,28.6139,77.2090,india,united states of america,north america,38.9072,-77.0369,250623.48,3.018946e+07
2157,237,21,-15.8267,-47.9218,brazil,vietnam,asia,21.0285,105.8542,1029815.60,1.770509e+08
2162,237,100,28.6139,77.2090,india,vietnam,asia,21.0285,105.8542,119617.58,7.184092e+07
2173,237,231,38.9072,-77.0369,united states of america,vietnam,asia,21.0285,105.8542,1303747.40,1.739689e+08


In [13]:
# Use continent color palette for map
continent_palette = {
    'africa': '#4E79A7',           # Blue
    'asia': '#EDC948',             # Yellow
    'europe': '#E15759',           # Red
    'north america': '#76B7B2', # Light Blue
    'oceania': '#B07AA1',          # Purple
    'south america': '#59A14F',    # Green
}

In [20]:
# Create traces for connections
traces = []

for _, row in soya.iterrows():
    continent = row['Continent_target']
    source_country = row['Country_source']
    target_country = row['Country_target']
    amount = row['Value_tons']
    color = continent_palette.get(continent, '#808080')  # Default to gray if continent not in palette
    
    traces.append(go.Scattergeo(
        lon=[row['lon_source'], row['lon_target']],
        lat=[row['lat_source'], row['lat_target']],
        mode='lines',
        line=dict(width=2, color=color),
        opacity=0.7,
        hoverinfo='text',
        hovertext=f"{source_country} -> {target_country}: {amount:.2f} tons",
        showlegend=False
    ))

# Create the layout
layout = go.Layout(
    title='Soya Beans Top Contaminating Trades (2021)',
    showlegend=True,
    legend_title_text='Target Continent',
    geo=dict(
        projection_type='natural earth',
        showland=True,
        showcountries=True,
        showocean=True,
        countrywidth=0.5,
        landcolor='rgb(243, 243, 243)',
        oceancolor='rgb(204, 229, 255)',
        showcoastlines=True,
        coastlinewidth=0.5,
        coastlinecolor='rgb(90, 90, 90)'
    )
)

# Create the figure
fig = go.Figure(data=traces, layout=layout)

# Add a legend
for continent, color in continent_palette.items():
    fig.add_trace(go.Scattergeo(
        lon=[None],
        lat=[None],
        mode='lines',
        line=dict(width=2, color=color),
        showlegend=True,
        name=continent,
    ))

# Show the figure
fig.show()
# fig.write_html("Soya_connections_map_plotly.html")
fig.write_image("Soya_connections_map_plotly.pdf")

## Save to chart studio

In [15]:
# import chart_studio 

# username = "ezemriv"
# password = "dmanBuRgSIAPjUFi4m1X"

# chart_studio.tools.set_credentials_file(username=username, api_key=password)

# import chart_studio.plotly as py
# import chart_studio.tools as tls

In [16]:
# py.plot(fig, filename = 'Soya_beans_top_contam_trades_final', auto_open=False)

# Map for Cattle (Livestock item with highest travel and emissions)

#961 non dairy

#960 dairy

In [17]:
cattle = process_map_data(trade_mx, 961)

In [18]:
# Create traces for connections
traces = []

for _, row in cattle.iterrows():
    continent = row['Continent_target']
    source_country = row['Country_source']
    target_country = row['Country_target']
    amount = row['Value_tons']
    color = continent_palette.get(continent, '#808080')  # Default to gray if continent not in palette
    
    traces.append(go.Scattergeo(
        lon=[row['lon_source'], row['lon_target']],
        lat=[row['lat_source'], row['lat_target']],
        mode='lines',
        line=dict(width=2, color=color),
        opacity=0.7,
        hoverinfo='text',
        hovertext=f"{source_country} -> {target_country}: {amount:.2f} tons",
        showlegend=False
    ))

# Create the layout
layout = go.Layout(
    title='Cattle (non-dairy) Top Contaminating Trades (2021)',
    showlegend=True,
    legend_title_text='Target Continent',
    geo=dict(
        projection_type='natural earth',
        showland=True,
        showcountries=True,
        showocean=True,
        countrywidth=0.5,
        landcolor='rgb(243, 243, 243)',
        oceancolor='rgb(204, 229, 255)',
        showcoastlines=True,
        coastlinewidth=0.5,
        coastlinecolor='rgb(90, 90, 90)'
    )
)

# Create the figure
fig = go.Figure(data=traces, layout=layout)

# Add a legend
for continent, color in continent_palette.items():
    fig.add_trace(go.Scattergeo(
        lon=[None],
        lat=[None],
        mode='lines',
        line=dict(width=2, color=color),
        showlegend=True,
        name=continent,
    ))

# Show the figure
fig.show()
# fig.write_html("Cattle_connections_map_plotly.html")

In [19]:
# py.plot(fig, filename = 'cattle_top_contam_trades_final', auto_open=False)